In [3]:
from transformers import pipeline
import matplotlib.pyplot as plt
import pandas as pd
from datasets import  load_metric
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
import nltk
import datasets
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
import pyarrow as pa
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\braim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
# tokenizer.bos_token = tokenizer.cls_token
# tokenizer.eos_token = tokenizer.sep_token
model = GPT2LMHeadModel.from_pretrained('distilgpt2').cuda()
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

loading file vocab.json from cache at C:\Users\braim/.cache\huggingface\hub\models--distilgpt2\snapshots\ee0b29d316a6a55fd15c50c4a97aca80ab4dd66a\vocab.json
loading file merges.txt from cache at C:\Users\braim/.cache\huggingface\hub\models--distilgpt2\snapshots\ee0b29d316a6a55fd15c50c4a97aca80ab4dd66a\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\braim/.cache\huggingface\hub\models--distilgpt2\snapshots\ee0b29d316a6a55fd15c50c4a97aca80ab4dd66a\config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0

In [71]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                        padding="longest", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8,max_new_tokens=30, num_beams=8)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [10]:
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")
train_data = pd.DataFrame(train_data)
val_data = pd.DataFrame(val_data)
test_data = pd.DataFrame(test_data)
training_articles = train_data['article']
test_mask = test_data['article'].str.len() <=1024
train_mask = train_data['article'].str.len() <=1024
val_mask = val_data['article'].str.len() <=1024
train_data = train_data.loc[train_mask]
val_data = val_data.loc[val_mask]
test_data = test_data.loc[test_mask]
validation_length = len(train_data.index)/10
if len(val_data.index) > validation_length:
    val_data = val_data.head(validation_length)
else:
    pass
if len(test_data.index) > validation_length:
    test_data = test_data.head(validation_length)
else:
    pass
train_data = datasets.Dataset(pa.Table.from_pandas(train_data))
val_data = datasets.Dataset(pa.Table.from_pandas(val_data))
test_data = datasets.Dataset(pa.Table.from_pandas(test_data))
print(len(train_data['article'][0]))

Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
Found cached dataset cnn_dailymail (C:/Users/braim/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


1009


In [ ]:
# dialogue_token_len = [len(tokenizer.encode(s) for s in train_data['article'])]
# summary_token_len = [len(tokenizer.encode(s) for s in train_data['highlights'])]
# fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey='C0' )
# axes[0].hist(dialogue_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
# axes[0].set_title("Dialogue Token Length")
# axes[0].set_xlabel("Length")
# axes[0].set_ylabel("Count")
# axes[1].hist(summary_token_len, bins = 20, color = 'C0', edgecolor = 'C0' )
# axes[1].set_title("Summary Token Length")
# axes[1].set_xlabel("Length")
# plt.tight_layout()
# plt.show()

In [63]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['article'], max_length = 1024, padding='max_length',truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['highlights'], max_length = 1024, padding='max_length',truncation = True )
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
train_data_pt = train_data.map(convert_examples_to_features, batched = True)
val_data_pt = val_data.map(convert_examples_to_features, batched = True)

  0%|          | 0/4 [00:00<?, ?ba/s]

C:\Users\braim\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [82]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [83]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='distilgpt', num_train_epochs=4, warmup_steps=0,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.1, logging_steps=10,
    evaluation_strategy='steps', eval_steps=100, save_steps=100,
    gradient_accumulation_steps=10,
    overwrite_output_dir=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [84]:
trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train_data_pt,
                  eval_dataset=val_data_pt)

In [36]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [85]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: article, __index_level_0__, highlights, id. If article, __index_level_0__, highlights, id are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
C:\Users\braim\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3670
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 10
  Total optimization steps = 1468
  Number of trainable parameters = 81914880


Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: article, __index_level_0__, highlights, id. If article, __index_level_0__, highlights, id are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 47
  Batch size = 1
Saving model checkpoint to distilgpt\checkpoint-100
Configuration saved in distilgpt\checkpoint-100\config.json
Model weights saved in distilgpt\checkpoint-100\pytorch_model.bin
tokenizer config file saved in distilgpt\checkpoint-100\tokenizer_config.json
Special tokens file saved in distilgpt\checkpoint-100\special_tokens_map.json
added tokens file saved in distilgpt\checkpoint-100\added_tokens.json
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: article, __index_level_0__, highlights, id. If article, __index_level_0__, highlights, id 

TrainOutput(global_step=1468, training_loss=0.35639616351686315, metrics={'train_runtime': 2063.8563, 'train_samples_per_second': 7.113, 'train_steps_per_second': 0.711, 'total_flos': 3835836298690560.0, 'train_loss': 0.35639616351686315, 'epoch': 4.0})

In [86]:
trainer.save_model('./distilgpt/finetuned_distilgpt2')

Saving model checkpoint to ./distilgpt/finetuned_distilgpt2
Configuration saved in ./distilgpt/finetuned_distilgpt2\config.json
Model weights saved in ./distilgpt/finetuned_distilgpt2\pytorch_model.bin
tokenizer config file saved in ./distilgpt/finetuned_distilgpt2\tokenizer_config.json
Special tokens file saved in ./distilgpt/finetuned_distilgpt2\special_tokens_map.json
added tokens file saved in ./distilgpt/finetuned_distilgpt2\added_tokens.json


In [87]:
model = GPT2LMHeadModel.from_pretrained('./distilgpt/finetuned_distilgpt2').cuda()
score = calculate_metric_on_test_ds(
    test_data, rouge_metric, model, tokenizer, batch_size = 1, column_text = 'article', column_summary= 'highlights'
)
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
pd.DataFrame(rouge_dict, index = ['trained distilgpt2'] )

loading configuration file ./distilgpt/finetuned_distilgpt2\config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "

,rouge1,rouge2,rougeL,rougeLsum
trained distilgpt2,0.004726,0.000305,0.004618,0.004683


In [80]:
print(train_data_pt[0])

{'article': "(CNN) -- The partnership started as a single shop on Oxford Street in London, opened in 1864 by John Lewis. Today the partnership is an organization with bases throughout the UK, with supermarkets and department stores, employing approximately 67,100 people. All 67,100 permanent staff are Partners who own 26 John Lewis department stores, 183 Waitrose supermarkets, an online and catalogue business, John Lewis Direct a direct services company - Greenbee, three production units and a farm. Every Partner receives the same scale of bonus, based on a fixed percentage of their annual wage. The bonus for 2006 was 18% equivalent to 9 weeks pay, which was rolled out for every employee. Chairman Sir Stuart Hampson retired at the end of March 2007, his successor is Charlie Mayfield. Hampson's salary for January 26, 2006 to January 26, 2007 was $1.66 million which included the partnership bonus of $250,000. John Lewis' consolidated revenue for the last financial year was $11.4 billion.